In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
data = load_diabetes()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10
)

print("train:", X_train.shape, "test:", X_test.shape)


train: (353, 10) test: (89, 10)


In [ ]:
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge())
])


# Explanation

// ...existing code...

This creates a machine that does two steps in order:

- Scale the features (StandardScaler)  
- Fit Ridge regression (Ridge)

Think: “data goes in → gets standardized → model learns coefficients”.

1) Why scaling at all?

Your dataset has multiple features (columns). They can be on different “units/scales”.

Example (toy):

- Feature A: “fat %” ~ 0.1–0.5  
- Feature B: “sodium mg” ~ 200–3000  
- Feature C: “temperature” ~ 5–50

If we fit a model using raw values, the feature with the biggest numerical scale can dominate optimization and regularization behavior.

StandardScaler does:

For each feature j:

$x'_{ij} = \dfrac{x_{ij} - \mu_j}{\sigma_j}$

where $\mu_j$ is the mean of feature $j$ (computed from training data) and $\sigma_j$ is the standard deviation of feature $j$.

After scaling:

- every feature has mean ≈ 0  
- every feature has standard deviation ≈ 1

Intuition: Scaling makes features “comparable”, like converting everything into “how many standard deviations from the average.”

2) Why scaling matters specifically for Ridge

Ridge regression solves:

$\min_{\beta}\; \dfrac{1}{n}\|y - X\beta\|^2 + \alpha\|\beta\|^2$

The penalty $\|\beta\|^2 = \sum_{j=1}^p \beta_j^2$ assumes each coefficient $\beta_j$ is “measured fairly.” If one feature is 1000× larger than another, coefficients for that feature will be scaled down accordingly, which changes how the penalty affects different features. With scaling, Ridge treats each feature on the same footing.

3) Why we need a Pipeline (the leakage reason)

Correct evaluation rule: nothing about the test/validation data should influence training.

Bad practice (data leakage):

```python
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)   # <-- uses ALL data including test
```

This uses test data to compute $\mu$ and $\sigma$ and can make CV/test scores look overly optimistic.

What Pipeline guarantees:

```python
pipe = Pipeline([("scaler", StandardScaler()), ("ridge", Ridge())])
```

When running cross-validation, scikit-learn will, for each fold:
- Fit scaler on training-fold only → compute $\mu, \sigma$ from training part  
- Transform training-fold using those  
- Transform validation-fold using the same scaler (no refit)  
- Fit ridge on scaled training-fold  
- Evaluate on scaled validation-fold

So: no leakage.

4) What a Pipeline is doing mathematically

You can think of the pipeline as a composed function:

$f(x) = \text{Ridge}(\text{Scale}(x))$

Where Scale(x) = standardized x. Pipeline = “preprocessing + model” treated as one object.

5) What happens when you call .fit and .predict

pipe.fit(X_train, y_train):
- scaler.fit(X_train) → compute $\mu, \sigma$  
- scaler.transform(X_train) → $X'_{\text{train}}$  
- ridge.fit($X'_{\text{train}}$, y_train) → learn $\beta$

pipe.predict(X_test):
- scaler.transform(X_test) using $\mu, \sigma$ from training  
- ridge.predict($X'_{\text{test}}$)

Test data is never used to determine scaling.

6) Why we left Ridge() without alpha here

In Day 5 we will tune $\alpha$ with GridSearchCV. We will search over $\alpha \in \{10^{-3}, \dots, 10^{3}\}$ by passing different values to `ridge__alpha` so they are evaluated fairly.

// ...existing code...

# Gridsearch

In [3]:
param_grid = {
    "ridge__alpha": np.logspace(-3, 3, 13)  # 1e-3 ... 1e+3
}
param_grid


{'ridge__alpha': array([1.00000000e-03, 3.16227766e-03, 1.00000000e-02, 3.16227766e-02,
        1.00000000e-01, 3.16227766e-01, 1.00000000e+00, 3.16227766e+00,
        1.00000000e+01, 3.16227766e+01, 1.00000000e+02, 3.16227766e+02,
        1.00000000e+03])}

In [ ]:
gscv = GridSearchCV(
    estimator=pipe, #this means we are tuning the pipeline
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",  # maximize "negative RMSE"
    cv=5,
    n_jobs=-1,
    refit=True,
    return_train_score=True
)

gscv.fit(X_train, y_train)

print("Best params:", gscv.best_params_)
print("Best CV score (neg RMSE):", gscv.best_score_)
print("Best CV RMSE:", -gscv.best_score_)
